In [1]:
import pandas as pd
import numpy as np
import re
import json
import ollama
import chromadb
import time

=============================================
# Pipeline functions
=============================================

In [33]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)

    def interpret_brightness(brightness_level):
        if 0 <= brightness_level <= 100:
            return "dark"
        elif 100 < brightness_level <= 150:
            return "dim"
        elif 150 < brightness_level <= 500:
            return "normal lighting"
        elif 500 < brightness_level <= 1000:
            return "very bright"
        else:
            return "dark"

    def generate_description(row):
        brightness_description = interpret_brightness(row['BrightnessLevel'])
        # return f"{row['UserActivity']} in the {row['Location']} when it is {brightness_description} at {row['Timestamp']}."
        pd.set_option('display.max_colwidth', None)
        description = f"{row['UserActivity']} in the {row['Location']} at {row['Timestamp']} when it is {brightness_description}."
        return description

    df['Description'] = df.apply(generate_description, axis=1)

    # only use the lastest state
    df_latest = df.iloc[-1:]
    df_latest = df_latest['Description']
    df_latest = df_latest.to_string(index=False)
    # print(df_latest)

    # use last 10 states
    df_history = df.iloc[-10:]
    df_history = df_history['Description']
    df_history = df_history.to_string(index=False)
    # print(df_history)

    return df_latest, df_history

def extract_json_from_response(response_text):
    """
    Extracts the JSON object from the assistant's response.
    """
    import json

    brace_count = 0
    json_start = None

    for i, char in enumerate(response_text):
        if char == '{':
            if brace_count == 0:
                json_start = i
            brace_count += 1
        elif char == '}':
            brace_count -= 1
            if brace_count == 0 and json_start is not None:
                json_str = response_text[json_start:i+1]
                try:
                    # Try to parse the extracted string
                    json.loads(json_str)
                    return json_str
                except json.JSONDecodeError:
                    pass  # Continue searching for valid JSON
    raise ValueError("No valid JSON object found in the assistant's response.")


    
def LLM2(knowledge_base, df_latest, collection, df_history, light_status):

    # user_query = f"""
    # There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    # Tell me Which lights should be turned on/off while the user is {df_latest}? "
    # """ 

    user_query = f"""
    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. light5 is the main light in the dining room next to living room closely.
    Now the light status is {light_status}, recent user behaviors are: {df_history}tell me Which lights should be turned on/off while the user is {df_latest}? "
    """ 

    print("User Query:")
    print(user_query)

    query_response = ollama.embeddings(
        prompt=user_query,
        model="mxbai-embed-large"
    )

    results = collection.query(
        query_embeddings=[query_response["embedding"]],
        n_results= 10  # Retrieve top 10 relevant documents
    )

    retrieved_documents = results['documents'][0]  
    data = ' '.join(retrieved_documents)
    print("Retrieved Data:")
    print(data)

    final_prompt = f"""
    You are a smart home assistant.

    Based on the knowledge base entries:
    {data}

    Respond to the user's current situation:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
        "description": "{df_latest}",
        "light1": "on/off",
        "light2": "on/off",
        "light3": "on/off",
        "light4": "open/close",
        "light5": "on/off",

    }}
    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    assistant_response = output['response']

    print("Assistant's Response:")
    print(assistant_response)

    try:
        # Extract JSON code block if the assistant included any text before or after
        json_code = extract_json_from_response(assistant_response)

        # Parse the JSON string
        actions = json.loads(json_code)

        # Remove the "description" key if not needed
        actions.pop("description", None)

        print("\nExtracted Light Control Actions:")
        print(actions)

        return actions

    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        # Handle the error or return None
        return None



def LLM1(df_latest, user_feedback, actions, df_history, collection):
    print("User Feedback:")
    print(user_feedback)
    
    user_update = f"""
    You are a smart home assistant responsible for deciding if to update the lighting knowledge based on user manually changes to the light states. 
    The user manually change the current light status from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Considering recent user behaviors, there are some trasition phase, so decide if to update the knowledge base based on the user feedback and recent behaviors.
    """

    query_response = ollama.embeddings(
        prompt=user_update,
        model="mxbai-embed-large"
    )

    results = collection.query(
        query_embeddings=[query_response["embedding"]],
        n_results=8  # Retrieve top 10 relevant documents
    )

    retrieved_documents = results['documents'][0]  
    data = ' '.join(retrieved_documents)
    print("Retrieved Data:")
    print(data)

    final_prompt_update = f"""
    You are a smart home assistant.

    Based on the retrieved current knowledge base entries:
    {data}

    Consider current situation:
    {user_update}

    Decide if to update the knowledge base in the fixed json format:
    {{
    "Update": "yes/no"  
    }}

    Provide the updated knowledge base in the fixed json format:
    {{
        "{df_latest}:",
        "light1": "on/off",
        "light2": "on/off",
        "light3": "on/off",
        "light4": "open/close",
        "light5": "on/off",

    }}
    """

    ## LLM1
    output = ollama.generate(
    model="qwen2.5:14b-instruct",
    prompt=final_prompt_update
    )

    knowledge_base_text = output['response']
    # print(knowledge_base_text)
    return knowledge_base_text


def extract_json_objects(text):
    """
    Extracts all JSON objects from the text.
    """
    json_objects = []
    brace_stack = []
    in_json = False
    json_start = 0

    for i, char in enumerate(text):
        if char == '{':
            if not in_json:
                json_start = i
                in_json = True
            brace_stack.append('{')
        elif char == '}':
            if brace_stack:
                brace_stack.pop()
                if not brace_stack:
                    # Found a complete JSON object
                    json_str = text[json_start:i+1]
                    json_objects.append(json_str)
                    in_json = False
        else:
            continue

    return json_objects

def update_knowledge_base(knowledge_base, knowledge_base_update):
    code_block_pattern = r'```json\s*([\s\S]*?)```'
    json_blocks = re.findall(code_block_pattern, knowledge_base_update)

    if not json_blocks:
        # Try to find any JSON objects in the text
        # json_blocks = re.findall(r'\{(?:[^{}]|(?R))*\}', knowledge_base_update)
        json_blocks = extract_json_objects(knowledge_base_update)

    update_decision = None
    updated_kb = None

    for idx, json_str in enumerate(json_blocks):
        print(f"\nExtracted JSON String {idx+1}:")
        print(json_str)

        # Clean up the JSON string
        json_str = json_str.strip()
        json_str = re.sub(r',\s*}', '}', json_str)  # Remove trailing commas before }
        json_str = re.sub(r',\s*\]', ']', json_str)  # Remove trailing commas before ]

        # Parse the JSON string
        try:
            json_data = json.loads(json_str)

            if "Update" in json_data:
                update_decision = json_data.get("Update")
                print(f"\nParsed Update Decision: {update_decision}")

                json_data.pop("Update", None)

                if json_data:
                    updated_kb = json_data
                    print("\nParsed Updated Knowledge Base:")
                    print(json.dumps(updated_kb, indent=2))
            else:
                updated_kb = json_data
                print("\nParsed Updated Knowledge Base:")
                print(json.dumps(updated_kb, indent=2))
        except json.JSONDecodeError as e:
            print(f"\nFailed to parse JSON in block {idx+1}: {e}")

    # Process the extracted data
    if update_decision is not None:
        print(f"\nUpdate Decision: {update_decision}")
        if update_decision.lower() == "yes":
            if updated_kb is not None:
                # light_status = updated_kb.get("description")
                knowledge_base = knowledge_base + "\n" + json.dumps(updated_kb, indent=2)
                print("\nKnowledge base has been updated.")
            else:
                print("\nNo updated knowledge base provided.")
        else:
            print("\nKnowledge base remains unchanged.")
    else:
        print("\nThe 'Update' decision was not found in the LLM's output.")

    return knowledge_base, update_decision


def get_user_confidence():
    confidence_input = input("Please rate your confidence in the system on a scale from 0 to 1: ")
    try:
        confidence = float(confidence_input)
        if 0 <= confidence <= 1:
            return confidence
        else:
            print("Invalid input. Confidence should be between 0 and 1.")
            return get_user_confidence()
    except ValueError:
        print("Invalid input. Please enter a number between 0 and 1.")
        return get_user_confidence()


knowledge_base = """
    This is the preference of the user:
    Prefer a dimly lit while watching TV.
    Prefer a dimly lit environment while reading books.
    Prefer a bright environment while exercising.
    Prefer a bright environment while eating.
    Prefer a dimly lit environment while playing video games.
    Prefer a dark environment while lying. 
    Prefer a bright environment while exercising.
    Prefer a bright environment while housekeeping.
    Prefer a dimly lit environment while typing.
    Prefer a dimly lit environment while sitting.
    Prefer a bright environment while cooking.
    Prefer a bright environment while grooming.
"""

In [5]:
knowledge_base_json = """
{
  "Layout of lights in the living room": [
    {
      "light_id": "light1",
      "description": "Main light with 3 color temperatures: warm, neutral, and cool."
    },
    {
      "light_id": "light2",
      "description": "Dimming light above the TV."
    },
    {
      "light_id": "light3",
      "description": "Lamp on the table."
    },
    {
      "light_id": "light4",
      "description": "Curtain on the window."
    },
    {
      "light_id": "light5",
      "description": "Main light in the dining room next to the living room."
    }
    
  ],
  "users": [
    {
      "user_id": "user1",
      "name": "Richard",
      "preferences": [
        {
          "activity": "watching TV",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "reading",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "cooking",
          "preferred_lighting": "bright"
        },
        {
          "activity": "sitting",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "lying",
          "preferred_lighting": "dark"
        },
        {
          "activity": "typing",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "housekeeping",
          "preferred_lighting": "bright"
        },
        {
          "activity": "exercising",
          "preferred_lighting": "bright"
        },
        {
          "activity": "video gaming",
          "preferred_lighting": "dimly lit"
        },
        {
            "activity": "eating",
            "preferred_lighting": "bright"
        },
        {
            "activity": "grooming",
            "preferred_lighting": "bright"
        }
      ]
    },
    {
      "user_id": "user2",
      "name": "Di",
      "preferences": [
        {
          "activity": "watching TV",
          "preferred_lighting": "dark"
        },
        {
          "activity": "reading",
          "preferred_lighting": "dimly lit"
        },
        {
          "activity": "cooking",
          "preferred_lighting": "bright"
        },
        {
          "activity": "sitting",
          "preferred_lighting": "bright"
        },
        {
          "activity": "lying",
          "preferred_lighting": "dark"
        },
        {
          "activity": "typing",
          "preferred_lighting": "bright"
        },
        {
          "activity": "housekeeping",
          "preferred_lighting": "bright"
        },
        {
          "activity": "exercising",
          "preferred_lighting": "bright"
        },
        {
          "activity": "video gaming",
          "preferred_lighting": "dark"
        },
        {
            "activity": "eating",
            "preferred_lighting": "bright"
        },
        {
            "activity": "grooming",
            "preferred_lighting": "dimly lit"
        }
      ]
    },
    {
      "user_id": "user3",
      "name": "Mingyi",
      "preferences": [
        {
          "activity": "watching TV",
          "preferred_lighting": "bright"
        },
        {
          "activity": "reading",
          "preferred_lighting": "bright"
        },
        {
          "activity": "cooking",
          "preferred_lighting": "bright"
        },
        {
          "activity": "sitting",
          "preferred_lighting": "bright"
        },
        {
          "activity": "lying",
          "preferred_lighting": "dark"
        },
        {
          "activity": "typing",
          "preferred_lighting": "bright"
        },
        {
          "activity": "housekeeping",
          "preferred_lighting": "bright"
        },
        {
          "activity": "exercising",
          "preferred_lighting": "bright"
        },
        {
          "activity": "video gaming",
          "preferred_lighting": "dimly lit"
        },
        {
            "activity": "eating",
            "preferred_lighting": "bright"
        },
        {
            "activity": "grooming",
            "preferred_lighting": "bright"
        }
      ]
    }

  ]
}

"""

=============================================
# Pipeline Main
=============================================

In [35]:
documents = knowledge_base.split('\n\n')
documents = [' '.join(doc.split()) for doc in documents]

client = chromadb.Client()

try:
    collection = client.get_collection("docs")
    print("Collection already exists.")
except Exception as e:
    # If the collection does not exist, create it
    collection = client.create_collection(name="docs")
    print("Collection created.")

# store each document in a vector embedding database
for i, d in enumerate(documents):
    response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
    embedding = response["embedding"]
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[d]
    )


def main():
    global knowledge_base
    previous_df_latest = None

    user = "Richard"
    light_status = {
        "light1": "off",
        "light2": "off",
        "light3": "off",
        "light4": "close",
        "light5": "off"
    }

    while True:
        df_latest, df_history = load_data('simulated_data/reading.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = LLM2(knowledge_base, df_latest, collection, df_history, light_status)
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "off", "light3": "off", "light4": "close", "light5": "off"}
            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    # user_feedback = None
                    continue
            else:
                user_feedback = None

            if user_feedback:
                user_condidence = get_user_confidence()
                # if user_condidence < 0.5:
                #     # use RL agent to update the knowledge base
                #     print("RL agent to update the knowledge base")
                #     knowledge_base = rl_agent_update(knowledge_base, user_feedback, actions, df_history)

                # else:
                #     knowledge_base_update = LLM1(user_feedback, actions, df_history)
                #     knowledge_base = update_knowledge_base(knowledge_base, knowledge_base_update)
                #     print("\nUpdated Knowledge Base:")
                #     print(knowledge_base)
                # print(user_feedback)
                # knowledge_base_update = LLM1(user_feedback, actions, df_history, collection)
                knowledge_base_update = LLM1(
                    df_latest=df_latest,
                    user_feedback=user_feedback,
                    actions=actions,
                    df_history=df_history,
                    collection=collection)
                
                knowledge_base, update_decision = update_knowledge_base(knowledge_base, knowledge_base_update)
                print("\nUpdated Knowledge Base:")
                print(knowledge_base)

                if update_decision == "yes":
                    light_status = user_feedback
                    print("\nUpdated Light Status:")
                    print(light_status)
                else:
                    print("\nLight Status remains unchanged.")
                    light_status = actions

            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main()

Collection already exists.


Add of existing embedding ID: 0
Insert of existing embedding ID: 0
Add of existing embedding ID: 1
Insert of existing embedding ID: 1


description changed
housekeeping in the living room at 14:26:00 when it is very bright.
User Query:

    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. light5 is the main light in the dining room next to living room closely.
    Now the light status is {'light1': 'off', 'light2': 'off', 'light3': 'off', 'light4': 'close', 'light5': 'off'}, recent user behaviors are:      walking in the living room at 14:08:00 when it is very bright.
housekeeping in the living room at 14:10:00 when it is very bright.
housekeeping in the living room at 14:12:00 when it is very bright.
housekeeping in the living room at 14:14:00 when it is very bright.
housekeeping in the living room at 14:16:00 when it is very bright.
housekeeping in the living room at 14:18:00 when it is very bright.
housekeeping in the living room at 14:20:00 whe

KeyboardInterrupt: 

In [12]:
print(knowledge_base)


    This is the preference of the user:
    Prefer a dimly lit while watching TV.
    Prefer a dimly lit environment while reading books.
    Prefer a bright environment while exercising.
    Prefer a bright environment while eating.
    Prefer a dimly lit environment while playing video games.
    Prefer a dark environment while lying. 
    Prefer a bright environment while exercising.
    Prefer a bright environment while housekeeping.
    Prefer a dimly lit environment while typing.
    Prefer a dimly lit environment while sitting.
    Prefer a bright environment while cooking.
    Prefer a bright environment while grooming.



=============================================
# 0-Shot
=============================================

In [29]:
delete_query = '''
    This is the preference of user1 Richard:

    Richard likes a bright environment.

    Richard requires a dimly lit while watching TV.

    Richard requires a dimly lit environment while reading books.

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while eating.

    Richard requires a dimly lit environment while playing video games.

    Richard requires a dark environment while sleeping (lying). 

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while housekeeping.

    Richard requires a dimly lit environment while working (typing).

    Richard requires a dimly lit environment while sitting.

    Richard requires a bright environment while cooking.

    Richard requires a bright environment while grooming.
    '''

def zero_shot(user, df_latest, user_feedback, df_history, actions, light_status):
    print("User Feedback:")
    print(user_feedback)
    
    user_query = f"""
    {user} is the user who is living in this apartment. 

    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    Now the current light status is {light_status}.
    Tell me Which lights should be turned on/off while the user is {df_latest}? "
    Then based on the user feedback: {user_feedback} and recent user behaviors: {df_history}, decide which lights should be turned on/off.

    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Considering recent user behaviors, decide if to update the light status base based on the user feedback.
    """

    final_prompt = f"""
    You are a smart home assistant.

    Respond to the user's current situation:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "light1": "on/off",
        "light2": "on/off",
        ...
    }},
    ...
    }}
    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    assistant_response = output['response']

    print("Assistant's Response:")
    print(assistant_response)

    try:
        # Extract JSON code block if the assistant included any text before or after
        json_code = extract_json_from_response(assistant_response)

        # Parse the JSON string
        actions = json.loads(json_code)

        # Remove the "description" key if not needed
        actions.pop("description", None)

        print("\nExtracted Light Control Actions:")
        print(actions)

        return actions

    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        # Handle the error or return None
        return None
    

def main_zeroshot():
    global knowledge_base
    previous_df_latest = None
    user = "Richard"
    light_status = {
        "light1": "off",
        "light2": "off",
        "light3": "off",
        "light4": "close",
        "light5": "off"
    }

    while True:
        df_latest, df_history = load_data('simulated_data/reading.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = zero_shot(
                user = user,
                df_latest = df_latest,
                user_feedback = None,
                df_history = None,
                actions = None,
                light_status = light_status)
            
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "off", "light3": "off", "light4": "open", "light5": "off"}
        
            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            light_status = actions

            actions_feedback = zero_shot(
                user=user,
                df_latest=df_latest,
                user_feedback=user_feedback,
                df_history=df_history,
                actions=actions,
                light_status=actions)
            
            print("\nUpdated Action:")
            print(actions_feedback)

            light_status = actions_feedback

            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main_zeroshot()

description changed
housekeeping in the living_room at 14:15:00 when it is very bright.
User Feedback:
None
Assistant's Response:
{
    "1.": {
        "description": "housekeeping in the living_room at 14:15:00 when it is very bright.",
        "light1": "off",
        "light2": "off",
        "light3": "on",
        "light4": "close",
        "light5": "off"
    }
}

Extracted Light Control Actions:
{'1.': {'description': 'housekeeping in the living_room at 14:15:00 when it is very bright.', 'light1': 'off', 'light2': 'off', 'light3': 'on', 'light4': 'close', 'light5': 'off'}}

Suggested Action:
{'1.': {'description': 'housekeeping in the living_room at 14:15:00 when it is very bright.', 'light1': 'off', 'light2': 'off', 'light3': 'on', 'light4': 'close', 'light5': 'off'}}
User Feedback:
{'light1': 'off', 'light2': 'off', 'light3': 'off', 'light4': 'open', 'light5': 'off'}
Assistant's Response:
Based on Richard's recent behaviors and the current brightness, it seems that he prefers h

KeyboardInterrupt: 

=============================================
# Chain of Thought
=============================================

In [31]:
delete_query = '''
    This is the preference of user1 Richard:

    Richard likes a bright environment.

    Richard requires a dimly lit while watching TV.

    Richard requires a dimly lit environment while reading books.

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while eating.

    Richard requires a dimly lit environment while playing video games.

    Richard requires a dark environment while sleeping (lying). 

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while housekeeping.

    Richard requires a dimly lit environment while working (typing).

    Richard requires a dimly lit environment while sitting.

    Richard requires a bright environment while cooking.

    Richard requires a bright environment while grooming.
    '''

def cot(user, df_latest, user_feedback, df_history, actions, light_status):
    print("User Feedback:")
    print(user_feedback)
    
    user_query = f"""
    {user} is the user who is living in this apartment.
    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    Now the current light status is {light_status}.
    Tell me Which lights should be turned on/off while the user is {df_latest}? "
    Then based on the user feedback: {user_feedback} and recent user behaviors: {df_history}, decide which lights should be turned on/off.

    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Considering recent user behaviors, decide if to update the lights base based on the user feedback.

    Provide your reasoning step by step.
    """

    final_prompt = f"""
    You are a smart home assistant.

    Respond to the user's current situation:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "light1": "on/off",
        "light2": "on/off",
        ...
    }},
    ...
    }}
    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    assistant_response = output['response']

    print("Assistant's Response:")
    print(assistant_response)

    try:
        # Extract JSON code block if the assistant included any text before or after
        json_code = extract_json_from_response(assistant_response)

        # Parse the JSON string
        actions = json.loads(json_code)

        # Remove the "description" key if not needed
        actions.pop("description", None)

        print("\nExtracted Light Control Actions:")
        print(actions)

        return actions

    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        # Handle the error or return None
        return None
    

def main_cot():
    global knowledge_base
    previous_df_latest = None
    user =  "Richard"
    light_status = {
        "light1": "off",
        "light2": "off",
        "light3": "off",
        "light4": "close",
        "light5": "off"
    }

    while True:
        df_latest, df_history = load_data('simulated_data/reading.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = cot(
                user = user,
                df_latest = df_latest,
                user_feedback = None,
                df_history = None,
                actions = None,
                light_status = light_status)
            
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "off", "light3": "off", "light4": "open", "light5": "off"}
        

            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            actions_feedback = cot(
                user=user,
                df_latest=df_latest,
                user_feedback=user_feedback,
                df_history=df_history,
                actions=actions,
                light_status=actions)
            
            print("\nUpdated Action:")
            print(actions_feedback)

            light_status = actions_feedback

            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main_cot()

description changed
housekeeping in the living_room at 14:15:00 when it is very bright.
User Feedback:
None
Assistant's Response:
Since Richard is housekeeping during a time when it is very bright outside, the need for artificial lighting might be minimal. However, to ensure that he can see clearly and comfortably while cleaning, we should consider turning on lights that provide subtle illumination without being too harsh.

Given this context, here's an appropriate suggestion:

- light1: This main light with three color temperatures could be set to a neutral or slightly cool setting at low brightness.
- light3: The lamp on the table might provide focused lighting where Richard is working.
- Light2 and light4 are less relevant for housekeeping purposes in bright daylight conditions. Keeping them off will minimize distractions.

Based on these considerations, here's how we could adjust:

{
    "1.": {
        "description": "housekeeping in the living_room at 14:15:00 when it is very bri

KeyboardInterrupt: 

=============================================
# ReACT
=============================================

In [32]:
delete_query = '''
    This is the preference of user1 Richard:

    Richard likes a bright environment.

    Richard requires a dimly lit while watching TV.

    Richard requires a dimly lit environment while reading books.

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while eating.

    Richard requires a dimly lit environment while playing video games.

    Richard requires a dark environment while sleeping (lying). 

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while housekeeping.

    Richard requires a dimly lit environment while working (typing).

    Richard requires a dimly lit environment while sitting.

    Richard requires a bright environment while cooking.

    Richard requires a bright environment while grooming.
    '''

def react(user, df_latest, user_feedback, df_history, actions, light_status):
    print("User Feedback:")
    print(user_feedback)
    
    user_query = f"""

    {user} is the user who is living in this apartment.
    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    Now the current light status is {light_status}.
    Tell me Which lights should be turned on/off while the user is {df_latest}? "
    Then based on the user feedback: {user_feedback} and recent user behaviors: {df_history}, decide which lights should be turned on/off.

    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Considering recent user behaviors, decide if to update the lights base based on the user feedback.

    
    Determine which lights should be turned on/off while the user is **{df_latest}**, considering the user preferences, lighting setup, user feedback, and recent user behaviors.
    For each step, first provide your reasoning, then specify any action you would take (e.g., "Action: [describe action]").
    If you need to recall user preferences or recent behaviors, indicate that you are accessing that information.
    After reasoning and actions, provide the final light control actions in the fixed format.
    """

    final_prompt = f"""
    You are a smart home assistant.

    Respond to the user's current situation:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "light1": "on/off",
        "light2": "on/off",
        ...
    }},
    ...
    }}
    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    actions = output['response']

    print("Assistant's Response:")
    print(actions)

    return actions

def main_react():
    global knowledge_base
    previous_df_latest = None
    user = "Richard"
    light_status = {
        "light1": "off",
        "light2": "off",
        "light3": "off",
        "light4": "close",
        "light5": "off"
    }

    while True:
        df_latest, df_history = load_data('simulated_data/reading.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = react(
                user = user,
                df_latest = df_latest,
                user_feedback = None,
                df_history = None,
                actions = None,
                light_status = light_status)
            
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "off", "light3": "off", "light4": "open", "light5": "off"}
        

            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            actions_feedback = react(
                user=user,
                df_latest=df_latest,
                user_feedback=user_feedback,
                df_history=df_history,
                actions=actions,
                light_status=actions)
            
            print("\nUpdated Action:")
            print(actions_feedback)

            light_status = actions_feedback

            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main_react()

description changed
housekeeping in the living_room at 14:15:00 when it is very bright.
User Feedback:
None
Assistant's Response:
Given that Richard is housekeeping in the living room at 14:15 and it is very bright outside, he might not need much artificial light. However, turning on a lamp can provide localized lighting which may be useful for detailed tasks like dusting or cleaning small items.

- **light3 (lamp on table)**: Since it's very bright outside, overall room brightness isn't an issue, but having a lamp turned on could help with visibility during housekeeping activities.
  
There is no need to adjust the main light and TV dimming light since natural light provides sufficient illumination for general cleaning. The curtains are closed which might be beneficial if direct sunlight is causing glare or making it difficult to see.

Since there's no user feedback or recent behavior to consider, let's proceed with turning on just one lamp for additional localized lighting during hou

KeyboardInterrupt: 

=============================================
# RAG
=============================================

In [6]:
delete_query = '''
    This is the preference of user1 Richard:

    Richard likes a bright environment.

    Richard requires a dimly lit while watching TV.

    Richard requires a dimly lit environment while reading books.

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while eating.

    Richard requires a dimly lit environment while playing video games.

    Richard requires a dark environment while sleeping (lying). 

    Richard requires a bright environment while exercising.

    Richard requires a bright environment while housekeeping.

    Richard requires a dimly lit environment while working (typing).

    Richard requires a dimly lit environment while sitting.

    Richard requires a bright environment while cooking.

    Richard requires a bright environment while grooming.
    '''

def rag(user, knowledge_base, df_latest, collection, df_history, actions, user_feedback):
    print("User Feedback:")
    print(user_feedback)

    user_query = f"""
    {user} is the user who is living in this apartment. Based on the Layout of lights in the living room, 
    There are 5 lights in the living room. light1 in the main light which has 3 color temperatures, warm, neutral and cool. light2 is a dimming light above the TV. light3 is a lamp on the table. light4 a curtain on the window. Dining room is next to living room, light5 is the main light in the dining room.
    Tell me Which lights should be turned on/off while the user is {df_latest}? "
    Then based on the user feedback: {user_feedback} and recent user behaviors: {df_history}, decide which lights should be turned on/off.

    The user manually changed the light state from suggested action: {actions} to: {user_feedback}. The recent 10 user behaviors are {df_history}.
    Decide if to update the knowledge base based on the user feedback and recent behaviors.
    """

    print("User Query:")
    print(user_query)

    query_response = ollama.embeddings(
        prompt=user_query,
        model="mxbai-embed-large"
    )

    results = collection.query(
        query_embeddings=[query_response["embedding"]],
        n_results=10  # Retrieve top 3 relevant documents
    )

    retrieved_documents = results['documents'][0]  
    data = ' '.join(retrieved_documents)
    print("Retrieved Data:")
    print(data)

    final_prompt = f"""
    You are a smart home assistant.

    Based on the knowledge base entries:
    {data}

    Respond to the user's current situation, user feedback and recent user behaviors:
    {user_query}

    Only provide the light control actions in the fixed format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "lightx": "on/off",
        "lightx": "on/off",
        ...
    }},
    ...
    }}

    Decide if to update the knowledge base in the fixed json format:
    {{
    "Update": "yes/no"  
    }}

    Provide the updated knowledge base in the fixed json format:
    {{
    "1.": {{
        "description": "{df_latest}",
        "lightx": "on/off",
        "lightx": "on/off",
        ...
    }},
    ...
    }}

    """

    output = ollama.generate(
        # model="qwen2.5:7b",
        model ="qwen2.5:14b-instruct",
        prompt=final_prompt
    )

    actions = output['response']

    print("Assistant's Response:")
    print(actions)

    return actions


def main_rag():
    global knowledge_base
    previous_df_latest = None

    actions = None
    user_feedback = None

    user = "Richard"

    while True:
        df_latest, df_history = load_data('simulated_data.csv')

        # check if df_latest changed
        if df_latest != previous_df_latest:
            print("description changed")
            print(df_latest)
            
            actions = rag(
                user = user,
                knowledge_base = knowledge_base,
                df_latest=df_latest,
                collection=collection,
                df_history=df_history,
                actions=actions,
                user_feedback=user_feedback)
            print("\nSuggested Action:")
            print(actions)

            user_feedback_input = input("Enter user feedback as a JSON string (or leave empty): ")
            # format: {"light1": "off", "light2": "on", "light3": "off", "light4": "open", "light5": "off"}
            if user_feedback_input:
                try:
                    user_feedback = json.loads(user_feedback_input)
                except json.JSONDecodeError:
                    print("Invalid JSON input. Please try again.")
                    user_feedback = None
            else:
                user_feedback = None

            if user_feedback:
                user_condidence = get_user_confidence()

                print(user_feedback)
                # knowledge_base_update = LLM1(user_feedback, actions, df_history, collection)
                knowledge_base_update = rag(
                    user=user,
                    df_latest=df_latest,
                    user_feedback=user_feedback,
                    actions=actions,
                    df_history=df_history)
                
                knowledge_base = update_knowledge_base(knowledge_base, knowledge_base_update)
                print("\nUpdated Knowledge Base:")
                print(knowledge_base)


            previous_df_latest = df_latest
        else:
            print("description not changed")
        
        time.sleep(10)

if __name__ == "__main__":
    main_rag()